In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [30]:
import pandas as pd 

In [33]:
cleaned_tweets = pd.read_csv('cleaned_dataset.csv',names=["text"])

In [34]:
cleaned_tweets.head()

,text
0,frankreich disasi mbappe christmas infantino f...
1,schmuckstücke für glanzvolle zeiten die elegan...
2,ein trauriger abend für kingsley
3,und sehen sich nach dem verlorenen endspiel de...
4,kolonialisiert den westen deutlich sichtbar ke...


In [35]:
cleaned_tweets = cleaned_tweets[cleaned_tweets['text'].notnull()]

In [36]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import nltk

In [18]:
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
german_stop_words = stopwords.words('german')

In [21]:

def summary(df, col):
    df['word_count'] =  df[col].apply(lambda x: len(x.split()))
    df['char_count'] = df[col].apply(lambda x : len(x))
    def average_word(x):
        words = x.split()
        return sum(len(word) for word in words) / len(words)
    df['average_word_length'] = df[col].apply(lambda x: average_word(x))
    df['stopword_count'] = df[col].apply(lambda x: len([word for word in x.split() if word.lower() in german_stop_words])) 
    df['stopword_rate'] = df.stopword_count / df.word_count
    return df

In [22]:
def plot_countplot(df, feature):
    ax = df[feature].value_counts().sort_index().plot(kind='bar')
    return ax

In [37]:
tweets = summary(cleaned_tweets, 'text')


In [38]:
mask = tweets[tweets.text.str.contains('var researchConfig')].index
tweets = tweets.drop(mask, axis=0)
tweets.sort_values(by='stopword_count').head()

,text,word_count,char_count,average_word_length,stopword_count,stopword_rate
0,frankreich disasi mbappe christmas infantino f...,17,120,6.117647,0,0.0
4513,we boycott the,3,14,4.000000,0,0.0
4520,natürlich,1,9,9.000000,0,0.0
2721,no go if its true,5,17,2.600000,0,0.0
2720,delays qatar world cup decision via,6,35,5.000000,0,0.0


In [39]:
tweets.describe()

,word_count,char_count,average_word_length,stopword_count,stopword_rate
count,6165.000000,6165.000000,6165.000000,6165.000000,6165.000000
mean,15.584266,96.770479,5.360789,6.821573,0.419424
std,9.274086,57.879146,1.173709,5.144311,0.184497
min,1.000000,1.000000,1.000000,0.000000,0.000000
25%,9.000000,53.000000,4.666667,3.000000,0.333333
50%,13.000000,84.000000,5.235294,6.000000,0.451613
75%,20.000000,125.000000,5.866667,10.000000,0.538462
max,51.000000,270.000000,24.333333,34.000000,1.000000


In [23]:
def createWordCloud(df, column):
    text = df[column].tolist()
    joined_text = " ".join(text)
    cloud = WordCloud(background_color="white").generate(joined_text)
    
    return cloud


In [40]:
pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=6913f12d952c00b517c523f1414dd6c5a261680ee24ad2915d69969a9ecc4da3
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [41]:

import os
import pandas as pd
from langdetect import detect_langs
from langdetect import detect_langs, LangDetectException
import numpy as np
# consisten results for language detection
from langdetect import DetectorFactory
DetectorFactory.seed = 0

In [42]:
# Detect text language
def detect_languages(df: pd.DataFrame, column: str) -> pd.Series:
    def detect_language(text: str) -> str:
        try:
            language = detect_langs(text)
        except LangDetectException as e:
            return "unknown"
        else:
            return language[0].lang
    return df[column].apply(detect_language)

In [49]:
language_series = detect_languages(cleaned_tweets,'text')


In [50]:
cleaned_tweets['language'] = language_series


In [51]:
cleaned_tweets.head(10)

,text,word_count,char_count,average_word_length,stopword_count,stopword_rate,language
0,frankreich disasi mbappe christmas infantino f...,17,120,6.117647,0,0.000000,no
1,schmuckstücke für glanzvolle zeiten die elegan...,9,74,7.333333,3,0.333333,de
2,ein trauriger abend für kingsley,5,32,5.600000,2,0.400000,de
3,und sehen sich nach dem verlorenen endspiel de...,11,85,6.818182,5,0.454545,de
4,kolonialisiert den westen deutlich sichtbar ke...,31,220,6.129032,13,0.419355,de
5,was forced to wear an islamic garment in his b...,39,202,4.205128,4,0.102564,en
6,müde und erschöpft zurück in deutschland aber ...,41,258,5.317073,19,0.463415,de
7,so haben borna sosa und seine kroaten wmplatz ...,12,69,4.833333,5,0.416667,de
8,wer hat eigentlich die drüben auf gewonnen,7,42,5.142857,3,0.428571,de
9,bei der geburt getrennt,4,23,5.000000,2,0.500000,de


In [52]:
German_data = cleaned_tweets[cleaned_tweets['language'] == 'de']


In [53]:
German_data.head(5)

,text,word_count,char_count,average_word_length,stopword_count,stopword_rate,language
1,schmuckstücke für glanzvolle zeiten die elegan...,9,74,7.333333,3,0.333333,de
2,ein trauriger abend für kingsley,5,32,5.600000,2,0.400000,de
3,und sehen sich nach dem verlorenen endspiel de...,11,85,6.818182,5,0.454545,de
4,kolonialisiert den westen deutlich sichtbar ke...,31,220,6.129032,13,0.419355,de
6,müde und erschöpft zurück in deutschland aber ...,41,258,5.317073,19,0.463415,de


In [54]:
German_data.count()

text                   5365
word_count             5365
char_count             5365
average_word_length    5365
stopword_count         5365
stopword_rate          5365
language               5365
dtype: int64

In [56]:
German_data.to_csv("german_tweets.csv",index=False)


In [10]:
from textblob import Word, TextBlob
from nltk.tokenize import word_tokenize
def preprocess(text, stopwords):                               
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    #after stemming, some words become hard to understand. For the interpretation purpose, the lemma is better than the linguistic root.
    words_filtered = [Word(word).lemmatize() for word in words_tokens_lower if word not in german_stop_words]

    preprocessed_text = " ".join(words_filtered)
    return preprocessed_text

In [57]:
tweets=German_data.copy()

In [59]:
tweets.describe()

,word_count,char_count,average_word_length,stopword_count,stopword_rate
count,5365.000000,5365.000000,5365.000000,5365.000000,5365.000000
mean,16.110158,100.644362,5.397780,7.648089,0.460394
std,9.055508,56.783744,1.097699,4.969362,0.143158
min,1.000000,3.000000,2.000000,0.000000,0.000000
25%,9.000000,59.000000,4.733333,4.000000,0.384615
50%,14.000000,87.000000,5.266667,7.000000,0.470588
75%,21.000000,126.000000,5.878788,10.000000,0.550000
max,51.000000,270.000000,24.333333,34.000000,1.000000


In [60]:
tweets.drop(columns=["word_count","char_count","average_word_length","stopword_count","stopword_rate"],inplace= True)

In [61]:
tweets.head(2)

,text,language
1,schmuckstücke für glanzvolle zeiten die elegan...,de
2,ein trauriger abend für kingsley,de


In [62]:
tweets.drop(columns=["language"],inplace=True)

In [63]:
first_300_rows = tweets[:300]

In [64]:
rest_of_data=tweets[301:]

In [65]:
rest_of_data.count()

text    5064
dtype: int64

In [ ]:
first_300_rows.to_csv('USWS/model_data.csv',index=False)

In [ ]:
rest_of_data.to_csv('USWS/rest_of_tweets.csv',index=False)

In [ ]:
first_300_rows['sentiment']=''

In [ ]:
first_300_rows

In [ ]:
labeled_data= pd.read_excel('USWS/labeled_data.xlsx')

In [ ]:
labeled_data.head(5)

In [ ]:
import re 

In [14]:
 import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
labeled_data['preprocessed_text'] = labeled_data.text.apply(lambda x: preprocess(x, german_stop_words))


In [67]:
labeled_data.head()

,Unnamed: 0,text,sentiment,preprocessed_text
0,0,schmuckstücke für glanzvolle zeiten die elegan...,positiv,schmuckstücke glanzvolle zeiten eleganten rosg...
1,1,ein trauriger abend für kingsley,negativ,trauriger abend kingsley
2,2,und sehen sich nach dem verlorenen endspiel de...,negativ,sehen verlorenen endspiel rassistischen anfein...
3,3,kolonialisiert den westen deutlich sichtbar ke...,negativ,kolonialisiert westen deutlich sichtbar kennze...
4,4,müde und erschöpft zurück in deutschland aber ...,positiv,müde erschöpft zurück deutschland immer geflas...


In [ ]:
#@title Standardtext für Titel
labeled_data.to_csv('processed_modell_data.csv')

In [ ]:
import pandas as pd
labeled_data=pd.read_csv('processed_modell_data.csv')


In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [ ]:
from keras.layers import Reshape


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
labeled_data=pd.read_csv('processed_modell_data.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into a training set (60%), validation set (20%), and test set (20%)
train_data, val_test_data, train_labels, val_test_labels = train_test_split(labeled_data['preprocessed_text'], labeled_data['sentiment'], test_size=0.4, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(val_test_data, val_test_labels, test_size=0.5, random_state=42)


In [ ]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers

# First Model with Test validate split

In [1]:
import pandas as pd
labeled_data=pd.read_csv('processed_modell_data.csv')

In [2]:
pip install keras.preprocessing.sequence


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement keras.preprocessing.sequence (from versions: none)
ERROR: No matching distribution found for keras.preprocessing.sequence


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from keras_preprocessing.sequence import pad_sequences


In [6]:
from torch.nn import CrossEntropyLoss
import torch
from transformers import BertForSequenceClassification, AdamW, BertTokenizer
from torch.utils.data import TensorDataset, random_split

# Load the BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-german-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

# Prepare the dataset
tweets = labeled_data.text # your tweets dataset
labels = labeled_data.sentiment.replace(['positiv', 'negativ', 'neutral'], [1, 0, 1])# the corresponding labels (0 for negative, 1 for positive)

# Tokenize the tweets and convert them to input tensors

input_ids = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in tweets]
attention_masks = [[1] * len(input_id) for input_id in input_ids]

# Set the maximum sequence length
MAX_LEN = 256

# Pad the input sequences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = pad_sequences(attention_masks, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create the input tensors
inputs = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels))

# Split the dataset into train and validation sets
train_size=80
train_inputs, validation_inputs = random_split(inputs, [train_size, len(inputs) - train_size])

#Set the batch size and number of epochs
batch_size = 32
num_epochs = 5

#Create the DataLoader for our training set
train_dataloader = torch.utils.data.DataLoader(train_inputs, batch_size=batch_size, shuffle=True)

#Create the DataLoader for our validation set
validation_dataloader = torch.utils.data.DataLoader(validation_inputs, batch_size=batch_size, shuffle=True)

#Create the Adam optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

#Create the loss function
criterion = CrossEntropyLoss()

#Set the device to run on (CPU or GPU)
device = torch.device( "cpu")

#Move the model and data to the device
model.to(device)
for inputs, attention_masks, labels in train_dataloader:
    inputs = inputs.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.to(device)


#Train the model
for epoch in range(num_epochs):
  for inputs, attention_masks, labels in train_dataloader:
    optimizer.zero_grad()
    outputs = model(inputs, attention_masks)
    loss = criterion(outputs[0], labels)
    loss.backward()
    optimizer.step()
  #Validate the model
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for inputs, attention_masks, labels in validation_dataloader:
      outputs = model(inputs, attention_masks)
      _,predicted = torch.max(outputs.logits.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      accuracy = correct / total
  print("Accuracy: {:.2f}%".format(accuracy * 100))

#Save the model
torch.save(model.state_dict(), "bert_model.pt")

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Accuracy: 73.18%
Accuracy: 73.18%
Accuracy: 75.00%
Accuracy: 74.55%
Accuracy: 75.45%


In [8]:
target_tweets=pd.read_csv('rest_of_tweets.csv')

In [8]:
target_tweets.head()

,text
0,team vom bei recherche über gastarbeiter kurzz...
1,und in so einem land soll die wm stattfinden w...
2,und ardfernsehteam bei recherchen zur situatio...
3,bei recherchearbeiten in katar zur fußballwm w...
4,obwohl und brav die tvrechte für die gekauft h...


In [24]:
target_tweets['text'] = target_tweets.text.apply(lambda x: preprocess(x, german_stop_words))


In [25]:
target_tweets.head()

,text
0,team recherche gastarbeiter kurzzeitig festgen...
1,land wm stattfinden eigentlich passieren krank...
2,ardfernsehteam recherchen situation gastarbeit...
3,recherchearbeiten katar fußballwm wurde zdf ka...
4,obwohl brav tvrechte gekauft


In [67]:
# Create an empty dataframe
df = pd.DataFrame()

# Add the tweet_text column to the dataframe
df['tweet_text'] = ["Das ist ein Postiv Tweet WM",'darf teilnehmerland hierauf antwort boycott wm.']

Predecton Function

In [5]:
import pandas as pd

def predict_sentiment(df,column_name):

  # Iterate over the rows in the dataframe
  for index, row in df.iterrows():
      # Encode the tweet text
      encoded_tweet = tokenizer.encode(row[column_name], add_special_tokens=True)
      input_ids = torch.tensor(encoded_tweet).unsqueeze(0)
      attention_masks = [[1] * len(input_id) for input_id in input_ids]
      attention_masks = torch.tensor(attention_masks)
      input_ids = input_ids.to(device)
      attention_masks = attention_masks.to(device)
      outputs = model(input_ids, attention_masks)
      _, predicted = torch.max(outputs.logits.data, 1)
      # Check if the predicted label is positive/neutral
      if predicted.item() == 1:
          predicted = "Postiv/neutral"
      else:
          predicted = "Negativ"
      # Add the sentiment column to the dataframe and assign the predicted value to the corresponding row
      df.at[index, 'sentiment'] = predicted
  return df


In [68]:
mm=predict_sentiment(df,'tweet_text')

In [69]:
mm.head()

,tweet_text,sentiment
0,Das ist ein Postiv Tweet WM,Postiv/neutral
1,darf teilnehmerland hierauf antwort boycott wm.,Postiv/neutral


In [70]:
predicted_tweets = predict_sentiment(target_tweets,'text')

In [77]:
predicted_tweets.head(10)

,text,sentiment
0,team recherche gastarbeiter kurzzeitig festgen...,Negativ
1,land wm stattfinden eigentlich passieren krank...,Postiv/neutral
2,ardfernsehteam recherchen situation gastarbeit...,Postiv/neutral
3,recherchearbeiten katar fußballwm wurde zdf ka...,Negativ
4,obwohl brav tvrechte gekauft,Postiv/neutral
5,belgischen zweitligisten übernommen spieler au...,Postiv/neutral
6,gewusst katar belgischen fußballclub ka eupen ...,Postiv/neutral
7,muhahahaha mindestens 5 jahre verschoben züge ...,Postiv/neutral
8,41 neue s 11600 zimmern gebaut gt via,Postiv/neutral
9,franz baukonzern wg zwangsarbeit zusammenhang ...,Postiv/neutral


# Second Modell Enhamced with Train, Valdiate, Test split

In [4]:
from torch.nn import CrossEntropyLoss
import torch
import numpy
from transformers import BertForSequenceClassification, AdamW, BertTokenizer
from torch.utils.data import TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras_preprocessing.sequence import pad_sequences


# Load the BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-german-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

# Prepare the dataset
tweets = labeled_data.text # your tweets dataset
labels = labeled_data.sentiment.replace(['positiv', 'negativ', 'neutral'], [1, 0, 1])# the corresponding labels (0 for negative, 1 for positive)


# Tokenize the tweets and convert them to input tensors

input_ids = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in tweets]
attention_masks = [[1] * len(input_id) for input_id in input_ids]

# Set the maximum sequence length
MAX_LEN = 256

# Pad the input sequences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = pad_sequences(attention_masks, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create the input tensors
inputs = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels))

# Split the dataset into train, validation, and test sets (80%/10%/10%)
train_inputs, temp_inputs = train_test_split(inputs, test_size=0.2)
val_inputs, test_inputs = train_test_split(temp_inputs, test_size=0.5)

#Set the batch size and number of epochs
batch_size = 32
num_epochs = 5

#Create the DataLoader for our training set
train_dataloader = torch.utils.data.DataLoader(train_inputs, batch_size=batch_size, shuffle=True)

#Create the DataLoader for our validation set
val_dataloader = torch.utils.data.DataLoader(val_inputs, batch_size=batch_size, shuffle=True)

#Create the DataLoader for our test set
test_dataloader = torch.utils.data.DataLoader(test_inputs, batch_size=batch_size, shuffle=True)

#Create the Adam optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

#Create the loss function
criterion = CrossEntropyLoss()

#Set the device to run on (CPU or GPU)
device = torch.device( "cpu")

#Move the model and data to the device
model.to(device)
for inputs, attention_masks, labels in train_dataloader:
    inputs = inputs.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.to(device)


#Train the model
for epoch in range(num_epochs):
  for inputs, attention_masks, labels in train_dataloader:
    optimizer.zero_grad()
    outputs = model(inputs, attention_masks)
    loss = criterion(outputs[0], labels)
    loss.backward()
    optimizer.step()
  #Validate the model
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    true_label = []
    pred_label = []
    for inputs, attention_masks, labels in val_dataloader:
      outputs = model(inputs, attention_masks)
      _,predicted = torch.max(outputs.logits.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      accuracy = correct / total
      true_label.extend(labels.cpu().numpy())
      pred_label.extend(predicted.cpu().numpy())
      print("Validation Accuracy: {:.2f}%".format(accuracy * 100))
      # Calculate the confusion matrix
      cm = confusion_matrix(true_label, pred_label, labels=[0,1])
      print("Validation Confusion Matrix: \n", cm)
     # Calculate the classification report
      cr = classification_report(true_label, pred_label, labels=[0, 1], target_names=["negative", "positive / neutral"])
      print("Validation Classification Report: \n", cr)
      # Calculate the f1 score 
      f1 = f1_score(true_label, pred_label, average='weighted')
      print("Validation F1 Score: {:.2f}".format(f1))


    correct = 0
    total = 0
    true_label = []
    pred_label = []
    for inputs, attention_masks, labels in test_dataloader:
      outputs = model(inputs, attention_masks)
      _,predicted = torch.max(outputs.logits.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      accuracy = correct / total
      true_label.extend(labels.cpu().numpy())
      pred_label.extend(predicted.cpu().numpy())
      print("Test Accuracy: {:.2f}%".format(accuracy * 100))
      # Calculate the confusion matrix
      cm = confusion_matrix(true_label, pred_label, labels=[0,1])
      print("Test Confusion Matrix: \n", cm)
     # Calculate the classification report
      cr = classification_report(true_label, pred_label, labels=[0,1], target_names=["negative", "positive / neutral"])
      print("Test Classification Report: \n", cr)
      # Calculate the f1 score 
      f1 = f1_score(true_label, pred_label, average='weighted')
      print("Test F1 Score: {:.2f}".format(f1))


#Save the model
torch.save(model.state_dict(), "bert_model.pt")

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Validation Accuracy: 73.33%
Validation Confusion Matrix: 
 [[ 0  8]
 [ 0 22]]
Validation Classification Report: 
                     precision    recall  f1-score   support

          negative       0.00      0.00      0.00         8
positive / neutral       0.73      1.00      0.85        22

          accuracy                           0.73        30
         macro avg       0.37      0.50      0.42        30
      weighted avg       0.54      0.73      0.62        30

Validation F1 Score: 0.62


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 80.00%
Test Confusion Matrix: 
 [[ 0  6]
 [ 0 24]]
Test Classification Report: 
                     precision    recall  f1-score   support

          negative       0.00      0.00      0.00         6
positive / neutral       0.80      1.00      0.89        24

          accuracy                           0.80        30
         macro avg       0.40      0.50      0.44        30
      weighted avg       0.64      0.80      0.71        30

Test F1 Score: 0.71


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 76.67%
Validation Confusion Matrix: 
 [[ 1  7]
 [ 0 22]]
Validation Classification Report: 
                     precision    recall  f1-score   support

          negative       1.00      0.12      0.22         8
positive / neutral       0.76      1.00      0.86        22

          accuracy                           0.77        30
         macro avg       0.88      0.56      0.54        30
      weighted avg       0.82      0.77      0.69        30

Validation F1 Score: 0.69
Test Accuracy: 83.33%
Test Confusion Matrix: 
 [[ 1  5]
 [ 0 24]]
Test Classification Report: 
                     precision    recall  f1-score   support

          negative       1.00      0.17      0.29         6
positive / neutral       0.83      1.00      0.91        24

          accuracy                           0.83        30
         macro avg       0.91      0.58      0.60        30
      weighted avg       0.86      0.83      0.78        30

Test F1 Score: 0.78
Validation Accurac

In [26]:
predicted_tweets = predict_sentiment(target_tweets,'text')

In [27]:
predicted_tweets.to_csv("predicted_tweets_enhanced.csv", index=False)

In [28]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [29]:
!cp bert_model_enahnced.pt "drive/My Drive/"